<center><h2>TPS June 2022</h2></center>
<center><h1>Regression-based Imputer</h1></center>

The idea is simple:

- Regress each feature agaist the rest, then predict, do this for all features, finally submit.

I expect to augment this notebook with other variants.

Do you have any suggestion?

# Simple Setup

In [ ]:
%reset -sf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Data

In [ ]:
from pandas import read_csv, DataFrame, Series, IndexSlice
#from pathlib import Path
#list(Path('/kaggle/input').rglob('*.*'))

data = read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv')
data, row_id = data.drop('row_id', axis=1), data['row_id']

# Reduce Memory

In [ ]:
"""from numpy import float64, float32, int64, int32, dtype

data.info()

def reduce_mem(df):
    df = df.copy()
    
    map_dtypes = {'int': dtype(int64), 'float': dtype(float32)}
    
    for col in df:
        if df[col].dtype == dtype(int64):
            df[col] = df[col].astype(int32)
        if df[col].dtype == dtype(float64):
            df[col] = df[col].astype(float32)
    return df

data = reduce_mem(data)
data.info()"""

In [ ]:
def reduce_memory(data, verbose=True):
    import numpy as np
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = data.memory_usage().sum() / 1024 ** 2
    for column in data.columns:
        column_type = data[column].dtypes
        if column_type in numerics:
            c_min = data[column].min()
            c_max = data[column].max()
            if str(column_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[column] = data[column].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[column] = data[column].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[column] = data[column].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[column] = data[column].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[column] = data[column].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[column] = data[column].astype(np.float32)
                else:
                    data[column] = data[column].astype(np.float64)
    end_mem = data.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Memory usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100*(start_mem - end_mem) / start_mem))
    return data

data = reduce_memory(data)

# Install

In [ ]:
# Patch Xeon Intel OneAPI Scikit accelerator
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

# Simple count

In [ ]:
# Count of missings

num_miss = data.isna().sum().sum()
num_miss

# Estimation

In [ ]:
# Estimation

from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import Lars
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(min_samples_leaf=5)
    #Lars(n_nonzero_coefs=20, normalize=False),
)

In [ ]:
# Impute

row_cols = []
values = []

print('col:', end=' ')
for i, col in enumerate(data):
    if data[col].isna().any():
        print(col, end='  ')
        # Fit non-na Y
        Y = data[col]
        idx = Y[~Y.isna()].index
        Y = Y.iloc[idx]
        X = data.drop(col, axis=1).iloc[idx, :]
        _ = pipe.fit(X, Y)
        
        # Predict na Y
        miss_idx = data[data[col].isna()].index
        pred = pipe.predict(data.drop(col, axis=1).iloc[miss_idx, :])
        
        # Persist
        row_cols.extend(Series(miss_idx.astype(str) + '-' + col).tolist())
        values.extend(Series(pred).tolist())
        
res = DataFrame({'row-col': row_cols, 'value': values})

In [ ]:
# Submit

sample_subm = read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv')
sample_subm = sample_subm.merge(res, left_on='row-col', right_on='row-col', how='outer')
sample_subm = sample_subm.drop('value_x', axis=1)
sample_subm.columns = ['row-col', 'value']
sample_subm.to_csv('submission.csv', index=False)
sample_subm

# ~ END ~